<a href="https://colab.research.google.com/github/MoisesTedeschi/COVID-19/blob/master/Vizualizando_dados_do_Covid_19_com_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VISUALIZANDO DADOS DO CORONAVÍRUS (COVID-19) COM PYTHON

In [0]:
import pandas as pd

In [0]:
# from google.colab import files
# files.upload()

In [3]:
file = 'data/covid_19_clean_complete.csv'
df = pd.read_csv(file, sep=',', parse_dates=['Date'])

ValueError: ignored

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
df.head(10)

In [0]:
df.info()

In [0]:
# Casos Ativos = Casos Confirmados - Mortes - Casos Recuperados
# Ou seja, pessoas não recuperadas, confirmadas e não mortas.
df['Active'] = df['Confirmed'] - df['Deaths'] - df['Recovered']

In [0]:
# Subistituindo Mainland China por China (Mesma Região, mas só vamos mudar o nome.)
df['Country/Region'] = df['Country/Region'].replace('Mainland China', 'China')

In [0]:
# Preenchendo "Missing Values" (Valores Nulos)
df[['Province/State']] = df[['Province/State']].fillna('')
df[['Confirmed', 'Deaths', 'Recovered', 'Active']] = df[['Confirmed', 'Deaths', 'Recovered', 'Active']].fillna(0)


In [0]:
# Convertendo datatypes para valores inteiros
df['Recovered'] = df['Recovered'].astype(int)

In [0]:
df.head()

## EXAMINANDO DADOS TEMPORAIS

In [0]:
df.Date.describe()

## AGRUPANDO DADOS

In [0]:
# Obtem o número de casos confirmados, mortes, recuperados e ativos agrupado por data e por região.
df_agrupado = df.groupby(['Date', 'Country/Region'])['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

In [0]:
# Ordena o dataframe por mais casos confirmados
df_agrupado.sort_values(by='Confirmed', ascending=False)

In [0]:
# Obtem o número de casos confirmados, mortes, recuperados e ativos agrupando por região.
df_group_paises = df.groupby('Country/Region')['Confirmed', 'Deaths', 'Recovered', 'Active'].sum().reset_index()

In [0]:
# ordena por países com mais casos confirmados
df_group_paises.sort_values(by='Confirmed', ascending=False)

In [0]:
# Agrupa quantidade de casos recuperados, mortes e ativos por data
temp = df.groupby('Date')['Recovered', 'Deaths', 'Active'].sum().reset_index()

In [0]:
# Remodela o dataframe com variável e valor para ter quantidades de recuperados, mortos e ativos
temp = temp.melt(id_vars="Date", value_vars=['Recovered', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')

In [0]:
temp.head(20)

## CASOS AO LONGO DO TEMPO


In [0]:
# Habilita modo offline do plot
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [0]:
# Definindo o renderizador
import plotly.io as pio
pio.renderers
pio.renderers.default = "colab"

In [0]:
# Cores
recuperados = '#21bf73'
mortes = '#ff2e63'
ativos = '#fe9801'

In [0]:
import plotly.express as px
fig = px.area(temp, 
              x="Date", 
              y="Count", 
              color='Case', 
              height=600,
              title='CASOS AO LONGO DO TEMPO',
              color_discrete_sequence = [recuperados, mortes, ativos])
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [0]:
import numpy as np

In [0]:
# Mapa de Choropleth é um mapa composto por polígonos coloridos. 
# É usado para representar variações espaciais de uma quantidade
fig = px.choropleth(df_agrupado,                                                   # casos agrupados por país
                    locations="Country/Region",                                    # definindo as regiões no mapa
                    locationmode='country names',                                  # define o modo de localização para todas regiões
                    color=np.log(df_agrupado["Confirmed"]),                        # define a cor pelo o valor de casos confirmados (aplica o log)
                    hover_name='Country/Region',                                   # define o texto interativo com o nome da região
                    hover_data=["Confirmed", "Deaths"],                            # define o texto interativo com o numero de casos confirmasos e mortes
                    animation_frame=df_agrupado["Date"].dt.strftime('%d-%m-%Y'),   # define o animate_frame com as datas
                    title='CASOS AO LONGO DO TEMPO',                               # define título
                    color_continuous_scale=px.colors.sequential.Magenta)           # define a paleta de cores
fig.update_layout(autosize=False, width=1200, height=800)                          # define tamanho da figura
fig.show()

## MORTES AO LONGO DO TEMPO

In [0]:
# Mapa de Choropleth é um mapa composto por polígonos coloridos. 
# É usado para representar variações espaciais de uma quantidade
fig = px.choropleth(df_agrupado,                                                   # casos agrupados por país
                    locations="Country/Region",                                    # definindo as regiões no mapa
                    locationmode='country names',                                  # define o modo de localização para todas regiões
                    color=np.log(df_agrupado["Deaths"]),                        # define a cor pelo o valor de casos confirmados (aplica o log)
                    hover_name='Country/Region',                                   # define o texto interativo com o nome da região
                    hover_data=["Confirmed", "Deaths"],                            # define o texto interativo com o numero de casos confirmasos e mortes
                    animation_frame=df_agrupado["Date"].dt.strftime('%d-%m-%Y'),   # define o animate_frame com as datas
                    title='MORTES AO LONGO DO TEMPO',                              # define título
                    color_continuous_scale=px.colors.sequential.Magenta)           # define a paleta de cores
fig.update_layout(autosize=False, width=1200, height=800)
fig.show()

In [0]:
!pip install plotly==4.5.2

In [0]:
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.express as px

In [0]:
# Filtra os dados considerando o último dia da base de dados
completo = df[df['Date'] == max(df['Date'])]

In [0]:
completo.head()

In [0]:
# Plota Painel                        
fig = px.treemap(completo.sort_values(by='Confirmed', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Confirmed",
                 height=600,
                 title='NÚMERO DE CASOS CONFIRMADOS',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()
# Plota Painel
fig = px.treemap(completo.sort_values(by='Deaths', ascending=False).reset_index(drop=True), 
                 path=["Country/Region", "Province/State"],
                 values="Deaths",
                 height=600,
                 title='NÚMERO DE MORTES CONFIRMADAS',
                 color_discrete_sequence = px.colors.qualitative.Dark2)
fig.data[0].textinfo = 'label+text+value'
fig.show()

## PICOS DE CASOS CONFIRMADOS E MORTES

In [0]:
fig = px.line(df_agrupado,
              x="Date",
              y="Confirmed",
              color='Country/Region',
              height=600,
              title='CASOS CONFIRMADOS',
              color_discrete_sequence = px.colors.qualitative.Dark2 )
fig.show()

fig = px.line(df_agrupado,
              x="Date",
              y="Deaths",
              color='Country/Region',
              height=600,
              title='MORTES CONFIRMADAS',
              color_discrete_sequence = px.colors.qualitative.Dark2)
fig.show()

## GRÁFICO COM FOLIUM

In [0]:
# Obtem os dados do último dia da base de dados
temp = df[df['Date'] == max(df['Date'])]

In [0]:
import folium

In [0]:
m = folium.Map(location=[0, 0], tiles='cartodbpositron',
               min_zoom=1, max_zoom=4, zoom_start=1)

for i in range(0, len(temp)):
    folium.Circle(
        location=[temp.iloc[i]['Lat'], temp.iloc[i]['Long']],
        color='crimson', fill='crimson',
        tooltip =   '<li><bold>Country : '+str(temp.iloc[i]['Country/Region'])+
                    '<li><bold>Province : '+str(temp.iloc[i]['Province/State'])+
                    '<li><bold>Confirmed : '+str(temp.iloc[i]['Confirmed'])+
                    '<li><bold>Deaths : '+str(temp.iloc[i]['Deaths']),
        radius=int(temp.iloc[i]['Confirmed'])**1.1).add_to(m)
m